In [1]:
from extract_contacts_and_attentions import *

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [2]:
warnings.simplefilter('ignore', PDBConstructionWarning)
parser = PDBParser()  

In [4]:
# # Get sequences from CASP7 file
# prot_data_dict = parse_casp7_file(casp_95)

# protein_id_counts = Counter(protein_id.split('_')[0] for protein_id in prot_data_dict)
# single_occurence_ids = [protein_id for protein_id, count in protein_id_counts.items() if count == 1]

# # protein_data = generate_fastas(single_occurence_ids)
# protein_data = load_fastas(fasta_dir)
# same_sequence_ids = check_casp_pdb_seqs(protein_data)
in_contact_sites, non_contact_sites, subset_non_contact_sites = contacts_per_pdb(same_sequence_ids,protein_data)
contact_data = generate_contact_data(in_contact_sites, subset_non_contact_sites)


In [8]:
seed_value = 72
random.seed(seed_value)
n_sequences = 300

sequence_ids = random.sample(same_sequence_ids, n_sequences)
# sequence_ids = same_sequence_ids
print(sequence_ids[:5])

['1VBL', '2CPZ', '1IUR', '1J32', '2C1C']


## Logisitic Regression

In [9]:
from sklearn.linear_model import LogisticRegression


In [10]:
X, y = output_x_y(sequence_ids, contact_data, protein_data)

Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.4, random_state=55)

In [12]:
log_reg_clf = LogisticRegression(random_state=55)
log_reg_clf.fit(X_train, y_train)

LogisticRegression(random_state=55)

In [13]:
y_pred = log_reg_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.87


In [14]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8702356706430027
Precision: 0.9447287615148413
Recall: 0.7913689625607316


In [15]:
from sklearn.ensemble import RandomForestClassifier


In [18]:
rf_clf = RandomForestClassifier(criterion='gini', max_depth=8,max_features='sqrt',n_estimators = 200,random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, n_estimators=200, random_state=42)

In [20]:
y_pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8927844050043643
Precision: 0.9009872241579558
Recall: 0.8868248070877394


In [21]:
from sklearn.svm import LinearSVC


In [22]:
linear_svc = LinearSVC(random_state=55)
linear_svc.fit(X_train, y_train)

/Users/williamharrigan/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC(random_state=55)

In [23]:
y_pred = linear_svc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8676171079429735
Precision: 0.933959101575595
Recall: 0.7962274935695913


In [25]:
contact_data['1UE8']

[{'res_1': 5,
  'res_2': 2,
  'sig_1': 'F',
  'sig_2': 'Y',
  'dist': 4.9943337,
  'in_contact': True},
 {'res_1': 8,
  'res_2': 5,
  'sig_1': 'M',
  'sig_2': 'F',
  'dist': 4.744529,
  'in_contact': True},
 {'res_1': 11,
  'res_2': 8,
  'sig_1': 'E',
  'sig_2': 'M',
  'dist': 4.98619,
  'in_contact': True},
 {'res_1': 12,
  'res_2': 8,
  'sig_1': 'S',
  'sig_2': 'M',
  'dist': 4.840004,
  'in_contact': True},
 {'res_1': 13,
  'res_2': 9,
  'sig_1': 'P',
  'sig_2': 'R',
  'dist': 4.986037,
  'in_contact': True},
 {'res_1': 21,
  'res_2': 16,
  'sig_1': 'W',
  'sig_2': 'Y',
  'dist': 4.395117,
  'in_contact': True},
 {'res_1': 22,
  'res_2': 15,
  'sig_1': 'N',
  'sig_2': 'Y',
  'dist': 4.920925,
  'in_contact': True},
 {'res_1': 23,
  'res_2': 14,
  'sig_1': 'L',
  'sig_2': 'V',
  'dist': 4.9677725,
  'in_contact': True},
 {'res_1': 29,
  'res_2': 26,
  'sig_1': 'C',
  'sig_2': 'Y',
  'dist': 4.951995,
  'in_contact': True},
 {'res_1': 35,
  'res_2': 32,
  'sig_1': 'D',
  'sig_2': 'V',

In [27]:
lr_clf = LogisticRegression(C=100, max_iter=100, penalty='none', solver='saga')
lr_clf.fit(X_train, y_train)

/Users/williamharrigan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/williamharrigan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/williamharrigan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=100, penalty='none', solver='saga')

In [28]:
y_pred = lr_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8779458830375327
Precision: 0.9340731070496083
Recall: 0.817947985138611
